In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output

###Lets start with Importing Libraries and data

In [2]:
import pandas as pd # to import csv and for data manipulation
import matplotlib.pyplot as plt # to plot graph
import numpy as np # for linear algebra
import datetime # to dela with date and time
%matplotlib inline
from sklearn.preprocessing import StandardScaler # for preprocessing the data
from sklearn.ensemble import RandomForestClassifier # Random forest classifier
from sklearn.tree import DecisionTreeClassifier # for Decision Tree classifier
from sklearn.svm import LinearSVC as SVC # for SVM classification
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split # to split the data
from sklearn.cross_validation import KFold # For cross vbalidation
from sklearn.model_selection import GridSearchCV # for tunnig hyper parameter it will use all combination of given parameters
from sklearn.model_selection import RandomizedSearchCV # same for tunning hyper parameter but will use random combinations of parameters
from sklearn.metrics import confusion_matrix,recall_score,precision_recall_curve,auc,roc_curve,roc_auc_score,classification_report
from imblearn.pipeline import make_pipeline
import warnings
warnings.filterwarnings('ignore')

/home/leshare/softwares/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
data = pd.read_csv("../data/real/page.dat",header = 0)

In [4]:
# now let us check in the number of Percentage
data['Class'][data['Class']=='positive'] = 1
data['Class'][data['Class']=='negative'] = 0
Count_Normal_transacation = len(data[data["Class"]==0]) 
Count_Fraud_transacation = len(data[data["Class"]==1]) 
Percentage_of_Normal_transacation = 1.*Count_Normal_transacation/(Count_Normal_transacation+Count_Fraud_transacation)
print("percentage of normal transacation is",Percentage_of_Normal_transacation*100)
Percentage_of_Fraud_transacation= 1.*Count_Fraud_transacation/(Count_Normal_transacation+Count_Fraud_transacation)
print("percentage of fraud transacation",Percentage_of_Fraud_transacation*100)

('percentage of normal transacation is', 89.7843567251462)
('percentage of fraud transacation', 10.215643274853802)


In [5]:
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.metrics import precision_recall_curve, auc, f1_score, precision_recall_fscore_support
from imblearn.metrics import geometric_mean_score

def evaluate_auc_prc(y, pred):
    precision, recall, thresholds = precision_recall_curve(y, pred)
    aucprc = auc(recall, precision)
    return aucprc


def evaluate_f1(y, y_pred, pos_label=1):
    precision, recall, f1, support = precision_recall_fscore_support(y, y_pred, pos_label=1)
    #print classification_report(y, y_pred)
    return f1[1]

def evaluate_kappa(y, y_pred):
    reward = confusion_matrix(y, y_pred)
    N = reward.sum()
    po = 1.*(reward[0][0]+reward[1][1])/N
    py = 1.*(reward[0][0]+reward[0][1])*(reward[0][0]+reward[1][0])/N/N
    pn = 1.*(reward[1][0]+reward[1][1])*(reward[0][1]+reward[1][1])/N/N
    pe = py+pn
    kappa = (po-pe)/(1-pe)
    return kappa

aucprc_score = make_scorer(evaluate_auc_prc, greater_is_better=True)

gms = make_scorer(geometric_mean_score, greater_is_better=True)

f1_scorer = make_scorer(evaluate_f1, greater_is_better=True)

kappa_scorer = make_scorer(evaluate_kappa, greater_is_better=True)

def data_split(x, test_size=0.4, seed=1): # preparing data for training and testing as we are going to use different data 
    #again and again so make a function
    x_features= x.ix[:,x.columns != "Class"]
    x_labels=x.ix[:,x.columns=="Class"]
    x_features_train,x_features_test,x_labels_train,x_labels_test = train_test_split(x_features,x_labels,
                                                                                     test_size=test_size, 
                                                                                     random_state=seed)
    x_features_train.index = np.arange(len(x_features_train))
    x_labels_train.index = np.arange(len(x_labels_train))
    x_features_test.index = np.arange(len(x_features_test))
    x_labels_test.index = np.arange(len(x_labels_test))
    print("length of training data")
    print(len(x_features_train))
    print("length of test data")
    print(len(x_features_test))
    return(x_features_train,x_features_test,x_labels_train,x_labels_test)

In [6]:
# before starting we should standridze 
from sklearn.preprocessing import RobustScaler
y = data['Class'].values
X = data.drop(['Class'], axis=1).values
scalar = RobustScaler()
X = scalar.fit_transform(X)
x_train = X
y_train = y.astype('int32')
idxes = np.arange(len(x_train))
np.random.seed(1234)
x_train = x_train[idxes]
y_train = y_train[idxes]

In [8]:
import pickle
pickle.dump({'x':x_train, 'y':np.eye(2)[y_train.astype('int32')]}, open('../data/real/page/train.pkl', 'wb+'))

In [7]:
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.linear_model import LogisticRegression as LR
from sklearn.svm import LinearSVC as SVM
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier as DT
from time import time

In [15]:
# Choose classifer with cross validation
# Supervised Model: LR, SVM (linear), SVM (rbf), DT, KNN, MLP
# Dataset: original
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import matthews_corrcoef

# LR
best_c = 0
best_reward = 0
for c in [1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]:
    clf = LR(C=c, random_state=0)
    reward = np.array(cross_val_score(clf, x_train, y_train, cv=5, n_jobs=5, scoring=make_scorer(matthews_corrcoef))).mean()
    if reward > best_reward:
        best_reward = reward
        best_c = c
print 'best_c for LR:{}'.format(best_c)
print 'best_reward for LR:{}'.format(best_reward)

# SVM (linear)
best_c = 0
best_reward = 0
for c in [1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]:
    clf = SVM(C=c, random_state=0)
    reward = np.array(cross_val_score(clf, x_train, y_train, cv=5, n_jobs=5, scoring=make_scorer(matthews_corrcoef))).mean()
    if reward > best_reward:
        best_reward = reward
        best_c = c
print 'best_c for SVM:{}'.format(best_c)
print 'best_reward for SVM:{}'.format(best_reward)

# SVM (rbf)
best_c = 0
best_reward = 0
for c in [1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]:
    clf = SVC(C=c, kernel='rbf', random_state=0)
    reward = np.array(cross_val_score(clf, x_train, y_train, cv=5, n_jobs=5, scoring=make_scorer(matthews_corrcoef))).mean()
    if reward > best_reward:
        best_reward = reward
        best_c = c
print 'best_c for SVC:{}'.format(best_c)
print 'best_reward for SVC:{}'.format(best_reward)

# DT 
best_c = 0
best_reward = 0
for c in [1, 2, 3, 4, 5, 6, 7, 8]:
    clf = DT(max_depth=c, random_state=0)
    reward = np.array(cross_val_score(clf, x_train, y_train, cv=5, n_jobs=5, scoring=make_scorer(matthews_corrcoef))).mean()
    if reward > best_reward:
        best_reward = reward
        best_c = c
print 'best_c for DT:{}'.format(best_c)
print 'best_reward DT:{}'.format(best_reward)

# KNN 
best_c = 0
best_reward = 0
for c in [1, 2, 3, 4, 5, 6, 7, 8]:
    clf = KNN(n_neighbors=c)
    reward = np.array(cross_val_score(clf, x_train, y_train, cv=5, n_jobs=5, scoring=make_scorer(matthews_corrcoef))).mean()
    if reward > best_reward:
        best_reward = reward
        best_c = c
print 'best_c for KNN:{}'.format(best_c)
print 'best_reward KNN:{}'.format(best_reward)

# MLP 
best_c = 0
best_reward = 0
for c in [1, 2, 3, 4, 5, 6, 7, 8]:
    clf = MLP(hidden_layer_sizes=[5, c], random_state=0)
    reward = np.array(cross_val_score(clf, x_train, y_train, cv=5, n_jobs=5, scoring=make_scorer(matthews_corrcoef))).mean()
    if reward > best_reward:
        best_reward = reward
        best_c = c
print 'best_c for MLP:{}'.format(best_c)
print 'best_reward MLP:{}'.format(best_reward)

best_c for LR:1.0
best_reward for LR:0.658655345844
best_c for SVM:1.0
best_reward for SVM:0.674569716744
best_c for SVC:10.0
best_reward for SVC:0.763492154926
best_c for DT:6
best_reward DT:0.763783766613
best_c for KNN:3
best_reward KNN:0.736692529755
best_c for MLP:7
best_reward MLP:0.765621885002


In [11]:
# MLP 
best_c = 0
best_reward = 0
for c in [1, 2, 3, 4, 5, 6, 7, 8]:
    clf = MLP(hidden_layer_sizes=[5, 5, c], random_state=0)
    reward = np.array(cross_val_score(clf, x_train, y_train, cv=5, n_jobs=5, scoring=gms)).mean()
    if reward > best_reward:
        best_reward = reward
        best_c = c
print 'best_c for MLP:{}'.format(best_c)
print 'best_reward MLP:{}'.format(best_reward)

best_c for MLP:8
best_reward MLP:0.862691026972


In [137]:
# Choose hyper-parameters with cross validation
# Supervised Model: SVM
# Dataset: original
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.svm import SVC
from sklearn.metrics import matthews_corrcoef

idxes = np.arange(len(x_train))
np.random.seed(1234)
np.random.shuffle(idxes)
best_c = 0
best_reward = 0
best_train_reward = 0
for c in [1e2]:
    clf = SVC(C=c, kernel='rbf', random_state=0, gamma=1e-2)
    rewards = cross_validate(clf, x_train[idxes], y_train[idxes], cv=5, n_jobs=5, 
                             scoring=make_scorer(matthews_corrcoef))
    train_reward = np.array(rewards['train_score']).mean()
    reward = np.array(rewards['test_score']).mean()
    if reward > best_reward:
        best_reward = reward
        best_train_reward = train_reward
        best_c = c
print 'best_c:{}'.format(best_c)
print 'best_reward:{}'.format(best_reward)
print 'best_train_reward:{}'.format(best_train_reward)

c:0.01
[ 0.83371375  0.86045259  0.86075788  0.84663623  0.79454812]
[ 0.89685119  0.88444035  0.89617762  0.88781452  0.90061675]
best_c:0.01
best_reward:0.839221713747
best_train_reward:0.893180085574


In [19]:
# Performance on original dataset with the chosen model.
clf = SVC(C=1e2, kernel='rbf',random_state=0, gamma=1e-2)
clf.fit(x_train, y_train)
preds = clf.predict(x_train)
reward = matthews_corrcoef(y_train, preds)
print 'reward on original dataset:{}'.format(reward)

reward on original dataset:0.897875942041


In [148]:
# Choose under-sampling ratio for different under-sampling methods
# Supervised Model: SVC(C=1e2, kernel='rbf',random_state=0, gamma=1e-2)
# Under sampling methods: Random, ENN, Cluster, TomekLinks, ALLKNN
from imblearn.under_sampling import (NearMiss, RandomUnderSampler, EditedNearestNeighbours, 
                                     CondensedNearestNeighbour, ClusterCentroids, TomekLinks,
                                     RepeatedEditedNearestNeighbours, AllKNN)
from imblearn.pipeline import make_pipeline
from collections import Counter


train_x = x_train
train_y = y_train
pos_num = (train_y == 1).sum()
neg_num = (train_y == 0).sum()

for sampler, name, time in zip([ClusterCentroids, RandomUnderSampler, EditedNearestNeighbours, TomekLinks, AllKNN],
                         ['Cluster', 'Random', 'ENN', 'TomekLinks', 'ALLKNN'], [1, 50, 1, 1, 1]):
    break
    max_i = 1
    best_reward = -1
    for i in np.arange(1, 1.0*neg_num/pos_num, 0.2):
        sample = sampler(ratio={0:int(i*pos_num)})
        train_x_s, train_y_s = sample.fit_sample(train_x, train_y)
        clf.fit(train_x_s, train_y_s)
        preds = clf.predict(train_x)
        reward = matthews_corrcoef(train_y, preds)
        if best_reward < reward:
            best_reward = reward    
            max_i = i
    print 'best under-sampling ratio with {}:{}'.format(name, max_i)
    if time == 0:
        print 'best reward with {}:{}'.format(name, best_reward)
        continue
    
    best_reward = 0
    for i in np.arange(time):
        sample = sampler(ratio={0:int(max_i*pos_num)})
        train_x_s, train_y_s = sample.fit_sample(train_x, train_y)
        clf.fit(train_x_s, train_y_s)
        preds = clf.predict(train_x)
        reward = matthews_corrcoef(train_y, preds)
        if best_reward < reward:
            best_reward = reward    
    print 'best reward with {}:{}'.format(name, best_reward)

In [21]:
# Perform EasyEnsemble and BalanceCascade
from imblearn.ensemble import EasyEnsemble, BalanceCascade

train_x = x_train
train_y = y_train
pos_num = (train_y == 1).sum()
neg_num = (train_y == 0).sum()

for sampler, name in zip([EasyEnsemble, BalanceCascade],
                         ['EasyEnsemble']):
    max_i = 1
    best_reward = -1
    for i in np.arange(1, 1.0*neg_num/pos_num, 0.2):
        sample = sampler(ratio={0:int(i*pos_num)}, replacement=False, n_subsets=10)
        train_x_s, train_y_s = sample.fit_sample(x_train, y_train)
        preds = None
        for x, y in zip(train_x_s, train_y_s):
            clf.fit(x, y)
            if preds is None:
                preds = clf.predict(train_x)
            else:
                preds += clf.predict(train_x)
        preds = (preds >= ((len(train_x_s)+1)/2)).astype('int32')
        reward = matthews_corrcoef(train_y, preds)
        if best_reward < reward:
            best_reward = reward    
            max_i = i
    print 'best under-sampling ratio for {}:{}'.format(name, max_i)
    print 'best reward for {}:{}'.format(name, best_reward)


best under-sampling ratio for EasyEnsemble:7.6
best reward for EasyEnsemble:0.903736955814


## First over sampling than down sampling

In [156]:
# Choose over-sampling ratio for different orver-sampling methods
# Supervised Model: SVM(C=1e2, kernel='rbf', random_state=0)
# Over sampling methods: Random, SMOTE, ADASYN
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.pipeline import make_pipeline
from collections import Counter

for sampler, name, time in zip([RandomOverSampler, SMOTE, ADASYN],
                         ['RandomOverSampler', 'SMOTE', 'ADASYN'], [10, 10, 10]):
    max_i = 1
    best_reward = -1
    for i in np.arange(1, 1.0*neg_num/pos_num, 0.5):
        sample = sampler(ratio={1:int(i*pos_num)})
        train_x_s, train_y_s = sample.fit_sample(train_x, train_y)
        clf.fit(train_x_s, train_y_s)
        preds = clf.predict(train_x)
        reward = matthews_corrcoef(train_y, preds)
        if best_reward < reward:
            best_reward = reward    
            max_i = i
    print 'best over-sampling ratio with {}:{}'.format(name, max_i)
    best_reward = 0
    bset_set = None
    for i in np.arange(time):
        sample = sampler(ratio={1:int(max_i*pos_num)})
        train_x_s, train_y_s = sample.fit_sample(train_x, train_y)
        clf.fit(train_x_s, train_y_s)
        preds = clf.predict(train_x)
        reward = matthews_corrcoef(train_y, preds)
        if best_reward < reward:
            best_reward = reward   
            bset_set = (train_x_s, train_y_s)
    print 'best reward with {}:{}'.format(name, best_reward)
    pickle.dump({'x':bset_set[0], 'y':np.eye(2)[bset_set[1].astype('int32')]}, open('../data/real/page/train_{}.pkl'.format(name), 'wb+'))
    

best over-sampling ratio with RandomOverSampler:2.0
best reward with RandomOverSampler:0.899958128481
best over-sampling ratio with SMOTE:1.5
best reward with SMOTE:0.90177586827
best over-sampling ratio with ADASYN:1.0
best reward with ADASYN:0.897875942041
